In [1]:
import numpy as np
import glob
import pandas as pd

In [2]:
#step 1 - combining all daily files into one file
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [3]:
# setting df as a DataFrame as combined_csv
#all data from the .csv is in strings it appears
df = pd.read_csv("combined_csv.csv")

#drop blanks from the ticker
df['ticker'] = df['ticker'].str.strip()
df['ticker'].replace('', np.nan, inplace=True)
df['ticker'].replace('TRUE', np.nan, inplace=True)
#drop ticker.str.len() > 6 from ticker (no ticker is longer than 6)
df['ticker'].values[df['ticker'].str.len() > 6,] = np.nan
df.dropna(subset=['ticker'], inplace=True)

#drop blanks and 0s from price
df['price'] = df['price'].str.strip() #strip blank spaces
df['price'].replace('', np.nan, inplace=True) #replace blanks with NaN
df['price'] = df['price'].replace(',','', regex=True) #strip commas
df['price'] = pd.to_numeric(df['price']) #convert price column from string to numbers
df['price'].replace(0,np.nan,inplace=True) #remove 0s
df.dropna(subset=['price'], inplace=True) #remove all NaNs

#converting num_mentions, pos_count, neg_count, price, price_change_net into int/floats
df['num_mentions'] = pd.to_numeric(df['num_mentions'])
df['pos_count'] = pd.to_numeric(df['pos_count'])
df['neg_count'] = pd.to_numeric(df['neg_count'])
df['price'] = pd.to_numeric(df['price'])
df['price_change_net'] = df['price_change_net'].replace(',','', regex=True) #strip commas
df['price_change_net'] = pd.to_numeric(df['price_change_net'])

#converting date column into date_time format
df['date'] = pd.to_datetime(df['date'])

#cleaning pct_mentions and converting to number
df['pct_mentions'] = df['pct_mentions'].str.strip() #strip blank spaces
df['pct_mentions'] = df['pct_mentions'].replace('<1','0.5', regex=True) #change '<1' to '0.5'
df['pct_mentions'] = pd.to_numeric(df['pct_mentions'])

In [4]:
df['price_change_pct'] = df['price_change_pct'].replace('%','', regex=True) #strip %
df['price_change_pct'] = df['price_change_pct'].replace(',','', regex=True) #strip ,
df = df.replace('nan%', np.nan).dropna(subset=['price_change_pct'])
# df['price_change_pct'] = df['price_change_pct'].str.strip() #strip blank spaces
# df['price_change_pct'] = pd.to_numeric(df['price_change_pct']) #convert to integer

In [5]:
df = df.set_index('ticker') #set index to ticker

In [6]:
df.to_csv('WSB.csv')

In [371]:
print(df.loc[[5290]])

     ticker       date                                      url  num_mentions  \
5290     OI 2021-01-19  https://finance.yahoo.com/quote/OI?p=OI             5   

     pct_mentions  pos_count  neg_count  bullish_pct  bearish_pct  \
5290           <1          0          0            0            0   

      neutral_pct  price  price_change_net price_change_pct  \
5290          100  12.69             -0.27            -2.08   

              time_of_price  
5290  At close:  4:00PM EST  


In [357]:
df.sort_values('price_change_pct')

,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
10453,LOGI,2021-02-05,https://finance.yahoo.com/quote/LOGI?p=LOGI,1,<1,0,0,0,0,100,109.70,109.70,+-0.06%,At close: 4:00PM EST
10434,XRT,2021-02-05,https://finance.yahoo.com/quote/XRT?p=XRT,1,<1,1,0,100,0,0,77.22,77.22,+-0.09%,At close: 3:59PM EST
10455,COST,2021-02-05,https://finance.yahoo.com/quote/COST?p=COST,1,<1,1,0,100,0,0,355.85,355.85,+-0.10%,At close: 4:00PM EST
10478,GEM,2021-02-05,https://finance.yahoo.com/quote/GEM?p=GEM,2,<1,0,0,0,0,100,40.69,40.69,+-0.17%,At close: 4:00PM EST
10474,BOX,2021-02-05,https://finance.yahoo.com/quote/BOX?p=BOX,1,<1,1,0,100,0,0,18.50,18.50,+-0.33%,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52033,WKHS,2021-09-22,https://finance.yahoo.com/quote/WKHS?p=WKHS,1,<1,0,1,0,100,0,8.19,0.00,0.00%,At close: 4:00PM EDT
26155,INDA,2021-04-12,https://finance.yahoo.com/quote/INDA?p=INDA,1,<1,0,1,0,100,0,41.69,0.00,0.00%,At close: 4:00PM EDT
28080,TDA,2021-04-22,https://finance.yahoo.com/quote/TDA?p=TDA,2,<1,0,2,0,100,0,25.58,0.00,0.00%,At close: 3:59PM EDT
35011,SHIP,2021-06-03,https://finance.yahoo.com/quote/SHIP?p=SHIP,2,<1,0,0,0,0,100,1.05,0.00,0.00%,At close: 4:00PM EDT


In [405]:
df.dtypes

ticker                      object
date                datetime64[ns]
url                         object
num_mentions                 int64
pct_mentions               float64
pos_count                    int64
neg_count                    int64
bullish_pct                  int64
bearish_pct                  int64
neutral_pct                  int64
price                      float64
price_change_net           float64
price_change_pct            object
time_of_price               object
dtype: object

In [435]:
df

,ticker,date,url,num_mentions,pct_mentions,pos_count,neg_count,bullish_pct,bearish_pct,neutral_pct,price,price_change_net,price_change_pct,time_of_price
0,GME,2020-12-28,https://finance.yahoo.com/quote/GME?p=GME,567,20.0,257,131,45,23,31,20.99,0.84,+4.17,At close: 4:00PM EST
1,PLTR,2020-12-28,https://finance.yahoo.com/quote/PLTR?p=PLTR,418,14.0,164,110,39,26,34,25.63,-2.12,-7.64,At close: 4:00PM EST
2,PSTH,2020-12-28,https://finance.yahoo.com/quote/PSTH?p=PSTH,80,2.0,51,19,63,23,12,25.99,-0.04,-0.15,At close: 4:00PM EST
3,PSA,2020-12-28,https://finance.yahoo.com/quote/PSA?p=PSA,3,0.5,0,0,0,0,100,229.14,0.29,+0.13,At close: 4:00PM EST
4,TD,2020-12-28,https://finance.yahoo.com/quote/TD?p=TD,9,0.5,2,1,22,11,66,56.04,-0.04,-0.07,At close: 4:00PM EST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54297,BNGO,2021-10-08,https://finance.yahoo.com/quote/BNGO?p=BNGO,1,0.5,0,0,0,0,100,5.18,0.22,+4.44,At close: 4:00PM EDT
54298,NNDM,2021-10-08,https://finance.yahoo.com/quote/NNDM?p=NNDM,1,0.5,0,0,0,0,100,5.68,0.16,+2.90,At close: 4:00PM EDT
54300,COIN,2021-10-08,https://finance.yahoo.com/quote/COIN?p=COIN,3,0.5,3,0,100,0,0,251.59,1.21,+0.48,At close: 4:00PM EDT
54301,ZM,2021-10-08,https://finance.yahoo.com/quote/ZM?p=ZM,1,0.5,1,0,100,0,0,256.27,2.62,+1.03,At close: 4:00PM EDT
